In [3]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

model = SentenceTransformer('all-MiniLM-L6-v2')

df = pd.read_csv('../data/filtered_events_country_code.csv', delimiter=',')
"""
#topics more expansive
topics = [
    "women rights", "health care", "farmers", "environment", "labor rights", "infrastructure", "palestine-israel conflict", "climate", "anti right wing", "immigration", "eviction","youth violence", "tourism",
    "unjust law enforcement", "democracy", "ukraine-russia war", "discrimination",
    "education", "housing", "culture", "social welfare", "political rights", "public services", "policital criticism", "animal welfare",
]
#topics strictly from slides
"""

"""
topics = [
    "pro palestine", 
    "pro israel",
    "climate",
    "consumerism",
    "migration",
    "labor rights",
    "racism",
    "higher education",
    "corruption",
    "anti-government",
    "peace"
]
"""

#with descriptions
topics = [
    "pro palestine: Content supportive of Palestinian rights, sovereignty, and resistance to Israeli occupation or military actions.",
    "pro israel: Content supportive of the state of Israel, its right to exist, defend itself, and maintain security.",
    "climate: Discussions related to climate change, global warming, environmental protection, and sustainability.",
    "consumerism: Criticism or commentary on materialism, overconsumption, capitalism, or commercial culture.",
    "migration: Topics about immigration, refugees, asylum seekers, border policies, and demographic movement.",
    "labor rights: Advocacy for workers' rights, fair wages, unions, labor laws, or critiques of workplace exploitation.",
    "racism: Issues related to racial discrimination, systemic injustice, white supremacy, or anti-racism advocacy.",
    "higher education: Content focused on universities, student protests, tuition fees, academic freedom, or campus politics.",
    "corruption: Allegations or discussions of government misconduct, bribery, embezzlement, or political scandals.",
    "anti-government: Expressions of dissent or protest against state power, authoritarianism, police violence, or national institutions.",
    "peace: Calls for ceasefires, nonviolence, diplomacy, coexistence, or reconciliation between opposing groups.",
    "lgbtq rights: Advocacy for the rights, safety, and equality of LGBTQ individuals, including issues of discrimination, marriage, identity, and visibility."
]

topic_embeddings = model.encode(topics, convert_to_tensor=True)

def assign_topic(text):
    note_embedding = model.encode(text, convert_to_tensor=True)
    cosine_scores = util.cos_sim(note_embedding, topic_embeddings)
    best_topic_idx = cosine_scores.argmax().item()
    return topics[best_topic_idx]
 
df['predicted_topic'] = df['clean_notes'].apply(assign_topic)

In [4]:
df = df[['notes', 'clean_notes', 'predicted_topic']]
df.to_csv("../data/notes_with_inferred_topics_ESS_strictly_slides.csv", index=False)